In [1]:
import pandas as pd
import re
import gensim
from nltk import sent_tokenize

In [2]:
file_path = "sample.txt"
f = open(file_path, 'r')
data = f.readlines()
f.close()

In [3]:
# stats
print('num lines: %s' %(len(data)))

num lines: 101


In [4]:
# parse text and create list of lists structure
# remove first whatsapp info message
dataset = data[1:]
cleaned_data = []

In [5]:
for line in dataset:
	# grab the info and cut it out
	date = line.split(",")[0]
	line2 = line[len(date):]
	time = line2.split("-")[0][2:]
	line3 = line2[len(time):]
	name = line3.split(":")[0][4:]
	line4 = line3[len(name):]
	message = line4[6:-1] # strip newline charactor
    #print(date, time, name, message)
	cleaned_data.append([date, time, name, message ])


In [6]:
# Create the DataFrame 
df = pd.DataFrame(cleaned_data, columns = [ 'Date', 'Time', 'Name', 'Message']) 

In [7]:

df.head()

,Date,Time,Name,Message
0,12/9/20,8:29 AM,+91 70089 40269,Hi bro
1,12/9/20,8:29 AM,+91 70089 40269,Have you done with ur project!
2,12/9/20,8:30 AM,Sushant,Not yet bro
3,12/9/20,8:30 AM,Sushant,I was confused about what to do in analysis part
4,12/9/20,8:31 AM,+91 70089 40269,He mentioned about Fibonacci heap or Fibonacci...


In [8]:
df.tail()

,Date,Time,Name,Message
95,12/11/20,5:03 PM,+91 70089 40269,Are you done with graph?
96,12/11/20,5:03 PM,Sushant,Not yet bro
97,12/11/20,5:04 PM,+91 70089 40269,Ok bro
98,12/11/20,5:04 PM,Sushant,Until then you strat with intro and formatting...
99,12/11/20,5:05 PM,+91 70089 40269,Ok bro


In [9]:
# Save it!
df.to_excel(r'sample.xlsx', index=False)

In [10]:
#Topic Modeling Begins from here
twb_data = pd.read_excel("sample.xlsx")

In [11]:
twb_data.head()

,Date,Time,Name,Message
0,12/9/20,8:29 AM,+91 70089 40269,Hi bro
1,12/9/20,8:29 AM,+91 70089 40269,Have you done with ur project!
2,12/9/20,8:30 AM,Sushant,Not yet bro
3,12/9/20,8:30 AM,Sushant,I was confused about what to do in analysis part
4,12/9/20,8:31 AM,+91 70089 40269,He mentioned about Fibonacci heap or Fibonacci...


In [12]:
twb_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Date     100 non-null    object
 1   Time     100 non-null    object
 2   Name     100 non-null    object
 3   Message  100 non-null    object
dtypes: object(4)
memory usage: 3.2+ KB


In [13]:
#Tokenization on only message column
data = twb_data.to_dict()
new_data = []
for j in data['Message'].values():
        if str(j) != 'nan':
            new_data.append(gensim.utils.simple_preprocess(j, deacc=True, min_len=3))
new_data

[['bro'],
 ['have', 'you', 'done', 'with', 'project'],
 ['not', 'yet', 'bro'],
 ['was', 'confused', 'about', 'what', 'analysis', 'part'],
 ['mentioned', 'about', 'fibonacci', 'heap', 'fibonacci', 'sequence'],
 ['you', 'know'],
 ['mailed', 'sir', 'about', 'and', 'told', 'this'],
 ['heaps'],
 ['bro'],
 ['what', 'other', 'things', 'you', 'asked'],
 ['how', 'much', 'completed', 'till', 'now'],
 ['was', 'doing', 'project', 'now', 'started', 'aads'],
 ['asked',
  'sir',
  'that',
  'need',
  'elaborate',
  'time',
  'complexity',
  'different',
  'operations',
  'compare',
  'both',
  'heaps',
  'sir',
  'said',
  'just',
  'run',
  'some',
  'input',
  'and',
  'make',
  'graph'],
 ['using', 'code'],
 ['yes', 'need', 'implement', 'codes', 'performing', 'this', 'operations'],
 ['yes'],
 ['output', 'will', 'graph'],
 [],
 ['have',
  'measure',
  'time',
  'for',
  'different',
  'outputs',
  'and',
  'from',
  'that',
  'time',
  'and',
  'elements',
  'manually',
  'plot',
  'the',
  'graph'

In [14]:
texts = new_data

In [15]:
#Creating Bigram of Tokened data
bigram = gensim.models.Phrases(new_data)

In [16]:
print(bigram)

Phrases<493 vocab, min_count=5, threshold=10.0, max_vocab_size=40000000>


In [18]:
from gensim.utils import lemmatize
from nltk.corpus import stopwords

In [19]:
#Getting nglish stop words
stops = set(stopwords.words('english'))

In [20]:
#Function for Stop word removal and Lemmatization
def process_texts(texts):
    texts = [[word for word in line if word not in stops] for line in texts]
    texts = [bigram[line] for line in texts]
    texts = [[word.decode("utf-8").split('/')[0] for word in lemmatize(' '.join(line), allowed_tags=re.compile('(NN)'), min_length=5)] for line in texts]
    return texts

In [21]:
#calling Function for Stop word removal and Lemmatization
train_texts = process_texts(new_data)

In [22]:
#Importing Lda and Dictionary library
from gensim.models import LdaModel
from gensim.models.wrappers import LdaMallet
from gensim.corpora import Dictionary

In [26]:
train_texts

[[],
 ['project'],
 [],
 ['analysis'],
 ['fibonacci', 'sequence'],
 [],
 [],
 [],
 [],
 ['thing'],
 ['completed'],
 ['project'],
 ['complexity', 'operation', 'input', 'graph'],
 [],
 ['operation'],
 [],
 ['output', 'graph'],
 [],
 ['measure', 'output', 'element', 'graph'],
 [],
 ['project', 'today'],
 [],
 [],
 [],
 ['point', 'report'],
 ['someone', 'something', 'share'],
 [],
 ['write', 'program', 'binomial', 'right'],
 [],
 ['graph'],
 [],
 ['think', 'operation', 'graph', 'format', 'element'],
 ['insertion', 'deletion'],
 ['thinking'],
 [],
 [],
 ['google', 'classroom'],
 [],
 [],
 ['graph'],
 ['report', 'tomorrow', 'graph', 'reference'],
 [],
 ['graph', 'paper', 'powerpoint', 'something'],
 ['excel'],
 ['report'],
 [],
 ['project'],
 [],
 ['project'],
 ['today'],
 [],
 [],
 ['report'],
 ['datum'],
 ['report'],
 [],
 [],
 ['operation', 'result', 'comparison', 'analysis', 'element', 'operation'],
 ['report', 'today', 'night'],
 [],
 [],
 ['insertion'],
 [],
 ['deletion'],
 ['operation

In [23]:
#Making corpus into Dictionary format, which is nothing but converting corpus into number matrix format 
dictionary = Dictionary(train_texts)
corpus = [dictionary.doc2bow(text) for text in train_texts]

In [24]:
print (corpus)

[[], [(0, 1)], [], [(1, 1)], [(2, 1), (3, 1)], [], [], [], [], [(4, 1)], [(5, 1)], [(0, 1)], [(6, 1), (7, 1), (8, 1), (9, 1)], [], [(9, 1)], [], [(7, 1), (10, 1)], [], [(7, 1), (10, 1), (11, 1), (12, 1)], [], [(0, 1), (13, 1)], [], [], [], [(14, 1), (15, 1)], [(16, 1), (17, 1), (18, 1)], [], [(19, 1), (20, 1), (21, 1), (22, 1)], [], [(7, 1)], [], [(7, 1), (9, 1), (11, 1), (23, 1), (24, 1)], [(25, 1), (26, 1)], [(27, 1)], [], [], [(28, 1), (29, 1)], [], [], [(7, 1)], [(7, 1), (15, 1), (30, 1), (31, 1)], [], [(7, 1), (18, 1), (32, 1), (33, 1)], [(34, 1)], [(15, 1)], [], [(0, 1)], [], [(0, 1)], [(13, 1)], [], [], [(15, 1)], [(35, 1)], [(15, 1)], [], [], [(1, 1), (9, 2), (11, 1), (36, 1), (37, 1)], [(13, 1), (15, 1), (38, 1)], [], [], [(26, 1)], [], [(25, 1)], [(9, 1)], [(9, 1)], [], [(9, 1), (39, 1)], [], [], [(19, 1)], [], [(40, 1)], [], [(0, 1)], [(41, 1), (42, 1), (43, 1)], [], [(44, 1)], [(43, 1), (45, 1)], [], [(46, 1)], [], [(7, 1)], [(34, 1)], [(11, 1)], [], [(9, 1), (47, 2)], [(7,

In [25]:
#Applying LDA Model on corpus
ldamodel = LdaModel(corpus=corpus, num_topics=2, id2word=dictionary)

In [26]:
ldamodel.show_topics()

[(0,
  '0.126*"graph" + 0.083*"report" + 0.082*"operation" + 0.035*"element" + 0.027*"insertion" + 0.026*"output" + 0.026*"point" + 0.025*"topic" + 0.021*"analysis" + 0.021*"project"'),
 (1,
  '0.079*"project" + 0.060*"operation" + 0.048*"graph" + 0.043*"deletion" + 0.037*"today" + 0.034*"something" + 0.033*"perform" + 0.031*"element" + 0.030*"excel" + 0.029*"report"')]

In [29]:
#Using Graphical representation Library and saving HLML file
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
visualisation = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.save_html(visualisation, 'LDA_Visualization.html')

/home/sushant/.local/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [32]:
pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

/home/sushant/.local/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=             x    y  topics  cluster      Freq
topic                                         
0      0.02562  0.0       1        1  64.91469
1     -0.02562  0.0       2        1  35.08531, topic_info=         Term      Freq     Total Category  logprob  loglift
13      today  2.000000  2.000000  Default  30.0000  30.0000
34      excel  1.000000  1.000000  Default  29.0000  29.0000
45     select  1.000000  1.000000  Default  28.0000  28.0000
16      share  1.000000  1.000000  Default  27.0000  27.0000
17    someone  1.000000  1.000000  Default  26.0000  26.0000
..        ...       ...       ...      ...      ...      ...
0     project  1.258935  4.703258   Topic2  -3.3375  -0.2706
11    element  0.800679  3.373209   Topic2  -3.7900  -0.3908
7       graph  1.375385  9.565683   Topic2  -3.2490  -0.8921
25   deletion  0.641854  2.698384   Topic2  -4.0111  -0.3887
26  insertion  0.522965  2.714951   Topic2  -4.2160  -0.5996

[101 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
1         1  0.986538    analysis
46        1  0.750947  assignment
19        1  0.502179    binomial
19        2  0.502179    binomial
44        1  0.774339      choose
...     ...       ...         ...
43        1  0.510503       topic
43        2  0.510503       topic
40        1  0.767511      ubuntu
40        2  0.767511      ubuntu
22        1  0.751427       write

[75 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2])

In [28]:
from gensim.models import CoherenceModel
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=ldamodel, texts=train_texts, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

/home/sushant/.local/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)

Coherence Score:  0.7221973163587267
